<h1>Tensorflow Datasets und Pipeline</h1>




Siehe: https://www.tensorflow.org/guide/data [Letzter Zugriff: 19.06.2024]

In [ ]:
# // Content Coming

Mit den Datasets von Tensorflow sind einige Dinge möglich, hier werden die Einzelheiten gezeigt.

In [211]:
import tensorflow as tf
import numpy as np
import pathlib
import os

<h2>Basics</h2>

Die Dataset Struktur von Tensorflow bietet viele Möglichkeiten für ETL. 

In [28]:
# Dataset:
data = np.array(range(20))
data

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19])

In [29]:
# Füge Dataset negative  und große Werte hinzu. 
data = np.append(data, [-10, -50, 200, 400])
data

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19, -10, -50, 200, 400])

In [30]:
# TF Dataset.:
tf_dataset = tf.data.Dataset.from_tensor_slices(data)
tf_dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

Durch dieses Dataset kann jetzt iteriert werden. <br>
Um wieder eine Zahl als Numpy zu bekommen, wird die Methode .numpy() angewendet.

Durch das einfache Iterieren kann man sich schnell die Daten anschauen.

In [23]:
for i in tf_dataset:
    #print(i)  # Ausgabe: tf.Tensor(0, shape=(), dtype=int32), ...
    print(i.numpy())  # Ausgabe: 0, ...

    # Oder direkt als Numpy mit: tf_dataset.as_numpy_iterator()

    # Oder nehme nur die ersten 5  mit tf_dataset.take(5):
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
-10
-50
-10
-50
200
400


Ein Dataset kann durch ein Lambda oder eine separate Funktion gefiltert werden. 

In [73]:
def print_dataset(dataset):
    for i in dataset.as_numpy_iterator():
        print(i)

def do_op(x):
    return x*3

In [43]:
# Lambda:
tf_dataset = tf_dataset.filter(lambda x: ( (x>0) & (x<70) ))

In [44]:
print_dataset(tf_dataset)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [76]:
# Wende weitere Operation an.
tf_dataset  = tf_dataset.filter(lambda x:  x>9)
tf_dataset2 = tf_dataset.map(lambda x: x*2)
tf_dataset3 = tf_dataset.map(do_op)  # Mit Funktion. 

In [72]:
print_dataset(tf_dataset2)

20
22
24
26
28
30
32
34
36
38


Die Daten können auch gemischt werden.

In [80]:
tf_dataset2_shuffled = tf_dataset2.shuffle(3)  # Um ganzes Dataset zu mischen, gebe Länge des Datasets an.
print_dataset(tf_dataset2_shuffled)

24
26
22
28
32
34
36
30
20
38


In [83]:
# Batching
for i in tf_dataset2.batch(2):
    print(i.numpy())

[20 22]
[24 26]
[28 30]
[32 34]
[36 38]


<h2>TF Input Pipeline</h2>

Als Pipeline geht alles in einer Zeile.

In [87]:
tf_dataset = tf.data.Dataset.from_tensor_slices(data)  # Erstelle Dataset.
# TF Input Pipeline:
# - Variable bei Lambda muss sich immer unterscheiden.
dataset = tf_dataset.filter(lambda x: ((x>0) & (x<70))).filter(lambda y: y>9).map(lambda z: z*2).shuffle(7).batch(2)

Jetzt wenden wir das ganze auf Bilder an.

In [88]:
# Hole Dataset
# - Siehe Tensorflow: https://www.tensorflow.org/datasets/catalog/tf_flowers
url = "http://download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=url, cache_dir='./data/datasets/tf_flowers', untar=True)

228813984/228813984 ━━━━━━━━━━━━━━━━━━━━ 20s 0us/step


Bei einem Verzeichnis, von wo wir die Bilder laden wollen, sind die Unterverzeichnisse mit den Klassen beschriftet. 


Verzeichnisstruktur: <br>
<i>tf_flowers/</i>:   Wo alle Bilder sind. <br>
<i>tf_flowers/<-Klasse->/ </i>: Klassen der Bilder.

In [110]:
name_dict = {'daisy':0, 'dandelion':1, 'roses':2, 'sunflowers':3, 'tulips':4 }

img_data = tf.data.Dataset.list_files('./data/datasets/tf_flowers/*/*')
img_data = img_dada.shuffle(len(img_dada))
type(img_data)  # Bilder noch nicht geladen.

tensorflow.python.data.ops.shuffle_op._ShuffleDataset

In [111]:
for i in img_data.take(7):
    print(i)

tf.Tensor(b'.\\data\\datasets\\tf_flowers\\tulips\\16265883604_92be82b973.jpg', shape=(), dtype=string)
tf.Tensor(b'.\\data\\datasets\\tf_flowers\\dandelion\\461632542_0387557eff.jpg', shape=(), dtype=string)
tf.Tensor(b'.\\data\\datasets\\tf_flowers\\roses\\14019883858_e5d2a0ec10_n.jpg', shape=(), dtype=string)
tf.Tensor(b'.\\data\\datasets\\tf_flowers\\sunflowers\\184683023_737fec5b18.jpg', shape=(), dtype=string)
tf.Tensor(b'.\\data\\datasets\\tf_flowers\\daisy\\5110107234_12ddc0206b_m.jpg', shape=(), dtype=string)
tf.Tensor(b'.\\data\\datasets\\tf_flowers\\tulips\\8628453641_6f87755815_m.jpg', shape=(), dtype=string)
tf.Tensor(b'.\\data\\datasets\\tf_flowers\\tulips\\14103897845_7986002615.jpg', shape=(), dtype=string)


Dann teilen wir die Daten auf in Train- und Testset.

In [162]:
# Z. B. so:
len_dataset = len(img_data)
train_set = img_data.take(int(len_dataset*0.8)) # Nehme x-Elemente.
test_set  = img_data.skip(int(len_dataset*0.8)) # Nehme die Elemente nach x-Elementen.

Für jede dieser Klassen brauchen wir ein Label. Dafür gibt es mehrere Wege. <br>

In [163]:
# So könnte man die Labels finden.
for i in img_data.take(3):
    print(tf.strings.split(i.numpy(), '\\')[4])

tf.Tensor(b'tulips', shape=(), dtype=string)
tf.Tensor(b'tulips', shape=(), dtype=string)
tf.Tensor(b'roses', shape=(), dtype=string)


In [164]:
len(train_set)

2936

In [175]:
# Gibt Label zurück.
def extract_label(path) :
    return tf.strings.split(path, '\\')[4] 
    
# Lade Bild und transformiere.
def load_img(path):
    label = extract_label(path)
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize(img, [150, 150])
    return img, label

def scale(img, label):
    return img/255, label

In [176]:
for img, label in train_set.map(load_img).take(1):
    print(f'Label: {label}\nimg: {img} \n')

Label: b'roses'
img: [[[ 68.011665  77.011665  84.011665]
  [ 71.085     79.305     86.695   ]
  [ 69.918335  78.78834   85.85333 ]
  ...
  [115.41501  129.        75.935005]
  [117.33     130.11      79.72    ]
  [114.10168  129.27168   75.881676]]

 [[ 65.        74.34      82.83    ]
  [ 63.095     73.265     81.755   ]
  [ 67.72333   76.918335  84.166664]
  ...
  [118.16666  128.16666   75.47498 ]
  [118.085    126.425     77.255   ]
  [118.05999  126.39999   75.51331 ]]

 [[ 64.10833   73.15833   82.15833 ]
  [ 65.45      74.5       83.5     ]
  [ 67.308334  77.941666  84.566666]
  ...
  [118.358315 128.13333   75.141655]
  [119.475    123.85      76.375   ]
  [118.1      121.20835   73.71663 ]]

 ...

 [[ 94.891655 138.86665   79.87499 ]
  [ 93.07498  134.07498   76.07498 ]
  [ 90.900024 134.90002   74.30001 ]
  ...
  [ 83.93332  111.43335   88.600006]
  [ 80.07498  102.57498   87.07498 ]
  [ 73.216675  89.92508   85.783325]]

 [[ 95.666664 136.66667   78.666664]
  [ 90.32999  13

Als Pipeline:

In [190]:
# Als Pipeline:
trainset = train_set.map(load_img).map(scale).take(2)

In [192]:
# Iteriere:
for img, label in trainset.take(2):
    print(f'Label: {label}\nimg: {img} \n')

Label: b'roses'
img: [[[0.9980065  0.9980065  0.9980065 ]
  [0.9968824  0.9968824  0.9968824 ]
  [0.9901634  0.9901634  0.9901634 ]
  ...
  [0.98984313 0.98984313 0.98984313]
  [0.9980392  0.9980392  0.9980392 ]
  [0.9993463  0.9993463  0.9993463 ]]

 [[0.99698037 0.99698037 0.99698037]
  [0.9941176  0.9941176  0.9941176 ]
  [0.99190855 0.99190855 0.99190855]
  ...
  [0.9912549  0.9912549  0.9912549 ]
  [0.99607843 0.99607843 0.99607843]
  [0.9993463  0.9993463  0.9993463 ]]

 [[0.99983656 0.99983656 0.99983656]
  [0.9998039  0.9998039  0.9998039 ]
  [0.99359477 0.99359477 0.99359477]
  ...
  [0.99607843 0.99607843 0.99607843]
  [0.99607843 0.99607843 0.99607843]
  [0.9993463  0.9993463  0.9993463 ]]

 ...

 [[0.9962419  0.9962419  0.9962419 ]
  [0.99980384 0.99980384 0.99980384]
  [0.99607843 0.99607843 0.99607843]
  ...
  [0.99607843 0.99607843 0.99607843]
  [0.99607843 0.99607843 0.99607843]
  [1.         1.         1.        ]]

 [[0.9993464  0.9993464  0.9993464 ]
  [0.99607843 0.